## 다음 문제를 풀이하시오.

diabetes.csv 파일을 대상으로 다음 문제를 풀이하시오.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("diabetes.csv")
df.head(2)

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0

### Q1. 당뇨 여부인 Outcome과 임신 횟수인 Pregnancies을 제외한 수치형 변수는  결측치가 0으로 기록이 되어 있다. 모든 변수의 결측치 개수를 더한 개수를 기술하시오.
(정답 예시: 123)

In [3]:
# df_na = df.drop(["Outcome", "Pregnancies"], axis = 1)
df_na = df.iloc[:, 1:-1]
df_na.head(1)

Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0      148             72             35        0  33.6   

   DiabetesPedigreeFunction  Age  
0                     0.627   50

In [4]:
ser = pd.Series([0, 2, 3, 0, 2])

In [5]:
np.where(ser == 0, np.nan, ser)

array([nan,  2.,  3., nan,  2.])

In [6]:
ser.replace(to_replace = 0, value = np.nan)

0    NaN
1    2.0
2    3.0
3    NaN
4    2.0
dtype: float64

In [7]:
sum(df_na["Glucose"] == 0)

5

In [8]:
x = df_na["Glucose"]
sum(x == 0)

5

In [9]:
df_na.apply(lambda x: sum(x == 0))

Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
dtype: int64

In [10]:
df_na.apply(lambda x: sum(x == 0)).sum()

652

### Q2. Outcome 변수를 제외한 변수간 상관분석을 실시하고 상관계수가 가장 높은 조합의 변수명과 상관계수를 기술하시오.
 - 변수명은 알파벳 순서대로 기술한다.
 - 상관계수는 반올림하여 소수점 셋째자리까지 기술한다.
 
(정답 예시: aaa, bbb, 0.123)

In [11]:
df_q2 = df.drop("Outcome", axis = 1).copy()
df_q2.head(1)

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   

   DiabetesPedigreeFunction  Age  
0                     0.627   50

In [12]:
df_q2.corr().round(2)

Pregnancies  Glucose  BloodPressure  SkinThickness  \
Pregnancies                      1.00     0.13           0.14          -0.08   
Glucose                          0.13     1.00           0.15           0.06   
BloodPressure                    0.14     0.15           1.00           0.21   
SkinThickness                   -0.08     0.06           0.21           1.00   
Insulin                         -0.07     0.33           0.09           0.44   
BMI                              0.02     0.22           0.28           0.39   
DiabetesPedigreeFunction        -0.03     0.14           0.04           0.18   
Age                              0.54     0.26           0.24          -0.11   

                          Insulin   BMI  DiabetesPedigreeFunction   Age  
Pregnancies                 -0.07  0.02                     -0.03  0.54  
Glucose                      0.33  0.22                      0.14  0.26  
BloodPressure                0.09  0.28                      0.04  0.24  
SkinThickness                0.44  0.39                      0.18 -0.11  
Insulin                      1.00  0.20                      0.19 -0.04  
BMI                          0.20  1.00                      0.14  0.04  
DiabetesPedigreeFunction     0.19  0.14                      1.00  0.03  
Age                         -0.04  0.04                      0.03  1.00

In [13]:
df_q2_corr = df_q2.corr()
type(df_q2_corr)

pandas.core.frame.DataFrame

In [14]:
df_q2_corr.apply(lambda x: x[x != 1].max())

Pregnancies                 0.544341
Glucose                     0.331357
BloodPressure               0.281805
SkinThickness               0.436783
Insulin                     0.436783
BMI                         0.392573
DiabetesPedigreeFunction    0.185071
Age                         0.544341
dtype: float64

In [15]:
df_q2_corr.apply(lambda x: x[x != 1].idxmax())

Pregnancies                           Age
Glucose                           Insulin
BloodPressure                         BMI
SkinThickness                     Insulin
Insulin                     SkinThickness
BMI                         SkinThickness
DiabetesPedigreeFunction          Insulin
Age                           Pregnancies
dtype: object

In [16]:
df_q2_corr_max = pd.DataFrame({"col":  df_q2_corr.apply(lambda x: x[x != 1].idxmax()),
                               "corr": df_q2_corr.apply(lambda x: x[x != 1].max())})
df_q2_corr_max = df_q2_corr_max.reset_index()
df_q2_corr_max

index            col      corr
0               Pregnancies            Age  0.544341
1                   Glucose        Insulin  0.331357
2             BloodPressure            BMI  0.281805
3             SkinThickness        Insulin  0.436783
4                   Insulin  SkinThickness  0.436783
5                       BMI  SkinThickness  0.392573
6  DiabetesPedigreeFunction        Insulin  0.185071
7                       Age    Pregnancies  0.544341

In [17]:
df_q2_corr_max.loc[df_q2_corr_max["corr"] == df_q2_corr_max["corr"].max(), ]

index          col      corr
0  Pregnancies          Age  0.544341
7          Age  Pregnancies  0.544341

### Q3. 임신 경험 여부와 BMI간의 관계를 보고자 한다. 이때 적절한 검정방법을 사용하고 해당 검정의 검정통계량의 절대값을 반올림하여 소수점 셋째 자리 까지 표기하시오.
(정답 예시: 1.234)

In [18]:
df_q3 = df.loc[:, ["Pregnancies", "BMI"]].copy()
df_q3.head(2)

Pregnancies   BMI
0            6  33.6
1            1  26.6

In [19]:
from scipy.stats import ttest_ind

In [20]:
stat, p = ttest_ind(df_q3.loc[df_q3["Pregnancies"] == 0, "BMI"],
                    df_q3.loc[df_q3["Pregnancies"] >  0, "BMI"])
round(abs(stat), 3)

3.341

### Q4. 임신 횟수, 혈당, 혈압, 피부 두께로 BMI를 설명할 수 있는지 확인해보려 한다. BMI를 종속변수로 한 다중선형 회귀분석을 실시하고 유의미한 변수의 개수와 결정계수를 기술하시오.
 - 결측치는 모두 제거하고 분석을 실시한다.
 - 결정계수는 소수점 둘 째 자리까지 기술한다.

(정답 예시: 1, 0.12)

In [21]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [22]:
df_q4 = df.loc[:, ["Pregnancies", "Glucose", "BloodPressure",
                   "SkinThickness", "BMI"]].copy()
df_q4.head(2)

Pregnancies  Glucose  BloodPressure  SkinThickness   BMI
0            6      148             72             35  33.6
1            1       85             66             29  26.6

In [23]:
len(df_q4)

768

In [24]:
for col in df_q4.columns[1:]:
    df_q4 = df_q4.loc[df_q4[col] != 0, :]
len(df_q4)

532

In [25]:
df_q4.apply(lambda x: sum(x == 0))

Pregnancies      77
Glucose           0
BloodPressure     0
SkinThickness     0
BMI               0
dtype: int64

In [26]:
df_q4 = df.loc[:, ["Pregnancies", "Glucose", "BloodPressure",
                   "SkinThickness", "BMI"]].copy()
df_q4.head(2)

Pregnancies  Glucose  BloodPressure  SkinThickness   BMI
0            6      148             72             35  33.6
1            1       85             66             29  26.6

In [27]:
# df_q4["Pregnancies"].replace() # to_replace, value
df_q4.apply("replace", to_replace = 0, value = np.nan).head()

Pregnancies  Glucose  BloodPressure  SkinThickness   BMI
0          6.0    148.0           72.0           35.0  33.6
1          1.0     85.0           66.0           29.0  26.6
2          8.0    183.0           64.0            NaN  23.3
3          1.0     89.0           66.0           23.0  28.1
4          NaN    137.0           40.0           35.0  43.1

In [28]:
df_q4 = pd.concat([df_q4["Pregnancies"], 
                   df_q4.iloc[:, 1:].apply("replace", to_replace = 0, value = np.nan)],
                  axis = 1)
df_q4.head()

Pregnancies  Glucose  BloodPressure  SkinThickness   BMI
0            6    148.0           72.0           35.0  33.6
1            1     85.0           66.0           29.0  26.6
2            8    183.0           64.0            NaN  23.3
3            1     89.0           66.0           23.0  28.1
4            0    137.0           40.0           35.0  43.1

In [29]:
df_q4.apply(lambda x: x.isna().sum())

Pregnancies        0
Glucose            5
BloodPressure     35
SkinThickness    227
BMI               11
dtype: int64

In [30]:
df_q4 = df_q4.dropna()

In [31]:
len(df_q4)

532

In [32]:
from statsmodels.formula.api import ols

In [33]:
formula = "BMI ~ " + " + ".join(df_q4.columns[:-1])
formula

'BMI ~ Pregnancies + Glucose + BloodPressure + SkinThickness'

In [34]:
model = ols(formula, data = df_q4).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    BMI   R-squared:                       0.461
Model:                            OLS   Adj. R-squared:                  0.456
Method:                 Least Squares   F-statistic:                     112.5
Date:                Fri, 28 Aug 2020   Prob (F-statistic):           2.86e-69
Time:                        10:31:34   Log-Likelihood:                -1616.3
No. Observations:                 532   AIC:                             3243.
Df Residuals:                     527   BIC:                             3264.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        12.9673      1.447      8.962      0.000      10.125      15.810
Pregnancies      -0.1960      0.068     -2.875      0.004      -0.330      -0.062
Glucose           0.0190      0.007      2.556      0.011       0.004       0.034
BloodPressure     0.0966      0.019      5.097      0.000       0.059       0.134
SkinThickness     0.3910      0.022     17.875      0.000       0.348       0.434
==============================================================================
Omnibus:                       42.530   Durbin-Watson:                   1.901
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              138.189
Skew:                           0.304   Prob(JB):                     9.83e-31
Kurtosis:                       5.422   Cond. No.                         964.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Q5. 결측치 처리에 따른 효과를 알아보고자 한다. 4번에서 사용된 독립변수 중 결측치를 모두 해당 변수의 평균치로 치환하고 다중선형 회귀분석을 실시하여 얻은 결정계수와 결측치를 연령대 평균 값으로 치환한 후 다중선형 회귀분석을 실시하여 얻은 결정계수를 차례대로 기술하시오.
 - 단, 이 분석은 70세 미만의 데이터만 사용한다.
 - 연령대 구분 단위는 10살 단위로 한다(17 => 1, 30 => 3, 45 => 4)
 - 결정계수는 소수점 둘 째 자리까지 기술한다.

(정답 예시: 0.12, 0.34)

In [104]:
df_q5 = df.loc[:, ["Pregnancies", "Glucose", "BloodPressure",
                   "SkinThickness", "BMI", "Age"]].copy()
df_q5 = df_q5.loc[df_q5["Age"] < 70, :"BMI"]
df_q5.head(2)

Pregnancies  Glucose  BloodPressure  SkinThickness   BMI
0            6      148             72             35  33.6
1            1       85             66             29  26.6

In [108]:
print(len(df), "=>", len(df_q5))

768 => 765


결측치를 평균치로 치환

In [109]:
pd.__version__

'0.25.1'

In [ ]:
# .apply(lambda x: x.replace(to_replace = 0, value = np.nan)) # ver. 0.23.4

In [36]:
df_q5_na = pd.concat([df_q5["Pregnancies"], 
                      df_q5.iloc[:, 1:].apply("replace",
                                              to_replace = 0, 
                                              value = np.nan)],
                      axis = 1)
df_q5_na.head(3)

Pregnancies  Glucose  BloodPressure  SkinThickness   BMI
0            6    148.0           72.0           35.0  33.6
1            1     85.0           66.0           29.0  26.6
2            8    183.0           64.0            NaN  23.3

In [37]:
df_q5_na_01 = df_q5_na.copy()
for col in df_q5_na_01.columns[1:]:
    df_q5_na_01[col] = df_q5_na_01[col].fillna(df_q5_na_01[col].mean())
    
df_q5_na_01.head(3)

Pregnancies  Glucose  BloodPressure  SkinThickness   BMI
0            6    148.0           72.0      35.000000  33.6
1            1     85.0           66.0      29.000000  26.6
2            8    183.0           64.0      29.166976  23.3

결측치를 연령대 평균치로 치환

In [38]:
df_q5_02 = df.loc[:, ["Pregnancies", "Glucose", "BloodPressure",
                      "SkinThickness", "BMI", "Age"]].copy()
df_q5_02 = df_q5_02.loc[df_q5_02["Age"] < 70, ]
df_q5_02.head(2)

Pregnancies  Glucose  BloodPressure  SkinThickness   BMI  Age
0            6      148             72             35  33.6   50
1            1       85             66             29  26.6   31

In [ ]:
# .apply(lambda x: x.replace(to_replace = 0, value = np.nan)) # ver. 0.23.4

In [39]:
df_q5_02 = pd.concat([df_q5["Pregnancies"], 
                      df_q5.iloc[:, 1:5].apply("replace", 
                                               to_replace = 0, 
                                               value = np.nan),
                      df_q5_02["Age"]],
                      axis = 1)
df_q5_02.head(3)

Pregnancies  Glucose  BloodPressure  SkinThickness   BMI  Age
0            6    148.0           72.0           35.0  33.6   50
1            1     85.0           66.0           29.0  26.6   31
2            8    183.0           64.0            NaN  23.3   32

In [40]:
df_q5_02["Age_group"] = df_q5_02["Age"] // 10 # 10으로 나눈 몫
df_q5_02.head(2)

Pregnancies  Glucose  BloodPressure  SkinThickness   BMI  Age  Age_group
0            6    148.0           72.0           35.0  33.6   50          5
1            1     85.0           66.0           29.0  26.6   31          3

In [110]:
df_q5_02.groupby("Age_group")[df_q5_02.columns[1:5]].mean()

Glucose  BloodPressure  SkinThickness        BMI
Age_group                                                     
2          114.613232      68.824468      27.769968  32.037629
3          126.073171      73.685897      30.620370  32.670122
4          125.709402      77.212389      31.706667  34.617797
5          140.280702      79.807018      30.125000  31.898214
6          138.827586      78.275862      34.545455  29.735714

In [41]:
group_mean = df_q5_02.groupby("Age_group")[df_q5_02.columns[1:5]].mean()
group_mean = group_mean.reset_index() # flat 하게 만듦, simplify
group_mean

Age_group     Glucose  BloodPressure  SkinThickness        BMI
0          2  114.613232      68.824468      27.769968  32.037629
1          3  126.073171      73.685897      30.620370  32.670122
2          4  125.709402      77.212389      31.706667  34.617797
3          5  140.280702      79.807018      30.125000  31.898214
4          6  138.827586      78.275862      34.545455  29.735714

In [42]:
for col in df_q5_02.columns[1:5]:
    for ag in df_q5_02["Age_group"].unique():
        print(col, ag)

Glucose 5
Glucose 3
Glucose 2
Glucose 4
Glucose 6
BloodPressure 5
BloodPressure 3
BloodPressure 2
BloodPressure 4
BloodPressure 6
SkinThickness 5
SkinThickness 3
SkinThickness 2
SkinThickness 4
SkinThickness 6
BMI 5
BMI 3
BMI 2
BMI 4
BMI 6


In [43]:
df_q5_02[col].isna().sum()

11

In [44]:
col = "SkinThickness"
ag = 3
cond_filter = (df_q5_02[col].isna()) & (df_q5_02["Age_group"] == ag)
df_q5_02.loc[cond_filter, ].head(2)

Pregnancies  Glucose  BloodPressure  SkinThickness   BMI  Age  Age_group
2            8    183.0           64.0            NaN  23.3   32          3
5            5    116.0           74.0            NaN  25.6   30          3

In [45]:
col = "SkinThickness"
ag = 3
repl_value = group_mean.loc[group_mean["Age_group"] == ag, col].values[0]
repl_value

30.62037037037037

In [46]:
for col in df_q5_02.columns[1:5]:
    for ag in df_q5_02["Age_group"].unique():
        # 결측치에 채워넣을 값
        repl_value = group_mean.loc[group_mean["Age_group"] == ag, col].values[0]
        
        # 필터링 조건
        cond_filter = (df_q5_02[col].isna()) & (df_q5_02["Age_group"] == ag)
        df_q5_02.loc[cond_filter, col] = repl_value
        
        print(col, ag, round(repl_value, 3))

Glucose 5 140.281
Glucose 3 126.073
Glucose 2 114.613
Glucose 4 125.709
Glucose 6 138.828
BloodPressure 5 79.807
BloodPressure 3 73.686
BloodPressure 2 68.824
BloodPressure 4 77.212
BloodPressure 6 78.276
SkinThickness 5 30.125
SkinThickness 3 30.62
SkinThickness 2 27.77
SkinThickness 4 31.707
SkinThickness 6 34.545
BMI 5 31.898
BMI 3 32.67
BMI 2 32.038
BMI 4 34.618
BMI 6 29.736


In [48]:
df_q5_02.head()

Pregnancies  Glucose  BloodPressure  SkinThickness   BMI  Age  Age_group
0            6    148.0           72.0       35.00000  33.6   50          5
1            1     85.0           66.0       29.00000  26.6   31          3
2            8    183.0           64.0       30.62037  23.3   32          3
3            1     89.0           66.0       23.00000  28.1   21          2
4            0    137.0           40.0       35.00000  43.1   33          3

df_q5_na_01, df_q5_02

종속변수: BMI, 독립변수는 4번의 것을 사용

In [51]:
formula = "BMI ~ " + " + ".join(df_q5_na_01.columns[:4])
formula

'BMI ~ Pregnancies + Glucose + BloodPressure + SkinThickness'

In [54]:
model_01 = ols(formula, data = df_q5_na_01).fit()
model_02 = ols(formula, data = df_q5_02).fit()

In [55]:
model_01.rsquared

0.3433107138481458

In [56]:
model_02.rsquared

0.337241373339354

### Q6. 분류모델의 성능을 비교하고자 한다. Outcome 변수를 종속변수로 하고 나머지 변수를 독립변수로 하고, 독립변수의 경우 결측치로 간주되는 것은 모두 제거한다. 이항 로지스틱 회귀분석과, 의사결정나무로 학습을 하고 그 정확도를 각각 기술하시오.
 - 학습은 최초 100개 데이터를 제외한 나머지로 실시한다.
 - 평가는 최초 100개 데이터로 실시한다.
 - 이항 로지스틱 회귀분석의 경우 유의미 하지 않은 변수는 제거한다.
 - 이항 로지스틱 회귀분석의 문턱값은 0.5로 설정한다.
 - 정확도는 반올림하여 소수점 셋째 자리까지 표기한다.
 
(정답 예시: 0.123, 0.456)

In [66]:
df_q6 = df.copy()
df_q6.head(2)

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0

In [67]:
df_q6.columns[1:-1]

Index(['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
       'DiabetesPedigreeFunction', 'Age'],
      dtype='object')

In [68]:
for col in df_q6.columns[1:-1]:
    df_q6 = df_q6.loc[df_q6[col] != 0, :]
len(df_q6)

392

In [69]:
df_q6 = df_q6.reset_index(drop = True)

In [70]:
df_q6_train = df_q6.iloc[100:, ] # 101 ~ 마지막
df_q6_test  = df_q6.head(100) # 처음 ~ 100

In [71]:
df_q6_train.head(2)

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
100            1       96             64             27       87  33.2   
101            0      140             65             26      130  42.6   

     DiabetesPedigreeFunction  Age  Outcome  
100                     0.289   21        0  
101                     0.431   24        1

In [72]:
from statsmodels.api import Logit
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [74]:
model_logit = Logit(df_q6_train["Outcome"],
                    df_q6_train.drop("Outcome", axis = 1)).fit()
model_logit.summary()

Optimization terminated successfully.
         Current function value: 0.538995
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                Outcome   No. Observations:                  292
Model:                          Logit   Df Residuals:                      284
Method:                           MLE   Df Model:                            7
Date:                Fri, 28 Aug 2020   Pseudo R-squ.:                  0.1234
Time:                        10:48:03   Log-Likelihood:                -157.39
converged:                       True   LL-Null:                       -179.54
Covariance Type:            nonrobust   LLR p-value:                 1.859e-07
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Pregnancies                  0.1348      0.059      2.280      0.023       0.019       0.251
Glucose                      0.0220      0.006      3.897      0.000       0.011       0.033
BloodPressure               -0.0505      0.012     -4.192      0.000      -0.074      -0.027
SkinThickness                0.0401      0.018      2.241      0.025       0.005       0.075
Insulin                      0.0018      0.001      1.323      0.186      -0.001       0.005
BMI                         -0.0594      0.028     -2.137      0.033      -0.114      -0.005
DiabetesPedigreeFunction     0.4404      0.409      1.077      0.282      -0.361       1.242
Age                         -0.0037      0.019     -0.194      0.846      -0.042       0.034
============================================================================================
"""

In [81]:
model_logit = Logit(df_q6_train["Outcome"],
                    df_q6_train.drop(["Outcome", "Age", "DiabetesPedigreeFunction",
                                      "Insulin", "BMI", "SkinThickness"],
                                     axis = 1)).fit()
model_logit.summary()

Optimization terminated successfully.
         Current function value: 0.553627
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                Outcome   No. Observations:                  292
Model:                          Logit   Df Residuals:                      289
Method:                           MLE   Df Model:                            2
Date:                Fri, 28 Aug 2020   Pseudo R-squ.:                 0.09961
Time:                        11:02:06   Log-Likelihood:                -161.66
converged:                       True   LL-Null:                       -179.54
Covariance Type:            nonrobust   LLR p-value:                 1.711e-08
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Pregnancies       0.1479      0.043      3.415      0.001       0.063       0.233
Glucose           0.0234      0.005      5.180      0.000       0.015       0.032
BloodPressure    -0.0587      0.009     -6.631      0.000      -0.076      -0.041
=================================================================================
"""

In [82]:
model_logit_sub = Logit(df_q6_train["Outcome"],
                        df_q6_train.drop(["Outcome", "Age", "DiabetesPedigreeFunction",
                                          "Insulin"],
                                         axis = 1)).fit()

Optimization terminated successfully.
         Current function value: 0.544159
         Iterations 6


In [85]:
model_tree = DecisionTreeClassifier()
model_tree.fit(X = df_q6_train.drop("Outcome", axis = 1),
               y = df_q6_train["Outcome"])
model_tree

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

model_logit, model_logit_sub, model_tree

In [98]:
pred_logit = model_logit.predict(df_q6_test.loc[:, ["Pregnancies", "BloodPressure", "Glucose"]])
pred_logit_sub = model_logit_sub.predict(df_q6_test.drop(["Outcome", "Age", "Insulin", 
                                                          "DiabetesPedigreeFunction"], axis = 1))
pred_tree = model_tree.predict(df_q6_test.drop("Outcome", axis = 1))

In [101]:
accuracy_score(y_true = df_q6_test["Outcome"],
               y_pred = (pred_logit > 0.5) + 0)

0.59

In [102]:
accuracy_score(y_true = df_q6_test["Outcome"],
               y_pred = (pred_logit_sub > 0.5) + 0)

0.68

In [103]:
accuracy_score(y_true = df_q6_test["Outcome"],
               y_pred = pred_tree)

0.68